In [1]:
import pandas as pd
import re

In [10]:
prof = pd.read_excel('dados_iniciais/professores_original.xlsx')
prof.to_csv('dados/docentes.csv',index=False)
prof.head()

,id_docente,docente,carga_maxima,max_disciplinas
0,docente_01,Amanda Cardoso Barros,200,1
1,docente_02,Bianca Nogueira Duarte,200,1
2,docente_03,Bruna Siqueira Tavares,200,1
3,docente_04,Bruno Cavalcante Moreira,200,1
4,docente_05,Camila Rodrigues Nunes,200,1


In [11]:
disc = pd.read_excel('dados_iniciais/disciplinas_original.xlsx')
disc.to_csv('dados/disciplinas.csv',index=False)
disc.head()

,id_disciplina,disciplina,carga_horaria,tipo_disciplina,codigo_turma,horario,horario_extenso
0,EST0001_01,COMPUTACAO EM ESTATISTICA 1,30,REGULAR,1,6M34,Sexta-feira 10:00 às 11:50
1,EST0001_02,COMPUTACAO EM ESTATISTICA 1,30,REGULAR,2,3T45,Terça-feira 16:00 às 17:50
2,EST0004_01,ESTATISTICA COMPUTACIONAL,60,REGULAR,1,3T2345,Terça-feira 14:00 às 17:50
3,EST0005_01,INFERENCIA BAYESIANA,60,REGULAR,1,35M34,Terça-feira 10:00 às 11:50Quinta-feira 10:00 à...
4,EST0011_01,MODELOS LINEARES GENERALIZADOS,60,REGULAR,1,24M34,Segunda-feira 10:00 às 11:50Quarta-feira 10:00...


In [7]:
pref = pd.read_excel('dados_iniciais/preferencias_original.xlsx')
pref.head()

,docente,EST0001_01,EST0001_02,EST0004_01,EST0005_01,EST0011_01,EST0017_01,EST0019_01,EST0019_02,EST0019_03,...,EST0064_01,EST0069_01,EST0069_03,EST0070_01,EST0081_01,EST0086_01,EST0088_01,EST0091_01,EST0092_01,EST0124_01
0,Amanda Cardoso Barros,2,2,2,3,2,2,2,0,0,...,3,3,2,1,0,1,1,0,2,0
1,Bianca Nogueira Duarte,3,0,0,2,0,1,1,2,2,...,3,3,1,2,1,1,1,3,3,2
2,Bruna Siqueira Tavares,0,1,2,3,0,2,2,0,2,...,1,1,0,2,0,2,0,2,0,3
3,Bruno Cavalcante Moreira,0,3,0,3,3,1,3,3,0,...,3,2,1,2,0,3,3,2,1,1
4,Camila Rodrigues Nunes,0,2,2,3,3,3,0,1,0,...,2,3,3,0,1,3,1,0,0,1


In [8]:
#Criando matriz de preferências
pref = pref.melt(id_vars='docente', var_name='id_disciplina', value_name='preferencia')
pref = pref.merge(prof, on = 'docente',how='left')[['id_docente','id_disciplina','preferencia']]
pref.to_csv('dados/preferencias.csv',index=False)
pref.head()

,id_docente,id_disciplina,preferencia
0,docente_01,EST0001_01,2
1,docente_02,EST0001_01,3
2,docente_03,EST0001_01,0
3,docente_04,EST0001_01,0
4,docente_05,EST0001_01,0


In [12]:
#Criando matriz de conflitos
def parse_horario(horario_str):
    """
    Interpreta uma string de horário no formato '24M12' e a converte
    em um dicionário estruturado.

    Args:
        horario_str (str): A string de horário (ex: '35T34', '6N12', etc.).

    Returns:
        dict: Um dicionário com os dias, turno e horários, ou None se o formato for inválido.
    """
    # Expressão regular para capturar os 3 componentes: dias, turno, blocos
    match = re.match(r'(\d+)([MTN])(\d+)', horario_str)
    
    if not match:
        return None # Formato inválido

    dias_str, turno_char, blocos_str = match.groups()

    mapa_dias = {'2': 'SEG', '3': 'TER', '4': 'QUA', '5': 'QUI', '6': 'SEX'}
    
    # Converte os dígitos dos dias em um conjunto de strings (ex: {'SEG', 'QUA'})
    dias_set = {mapa_dias[dia] for dia in dias_str if dia in mapa_dias}
    
    # Converte os dígitos dos blocos em um conjunto de inteiros (ex: {1, 2})
    blocos_set = {int(bloco) for bloco in blocos_str}

    return {
        'dias': dias_set,
        'turno': turno_char,
        'blocos': blocos_set
    }

def verificar_conflito(horario_str1, horario_str2):
    """
    Verifica se duas strings de horário entram em conflito.

    Args:
        horario_str1 (str): Horário da primeira disciplina.
        horario_str2 (str): Horário da segunda disciplina.

    Returns:
        bool: True se houver conflito, False caso contrário.
    """
    h1 = parse_horario(horario_str1)
    h2 = parse_horario(horario_str2)

    if h1 is None or h2 is None:
        print(f"Aviso: Formato de horário inválido encontrado ({horario_str1} ou {horario_str2})")
        return False

    # 1. Verifica se os turnos são diferentes (se forem, não há conflito)
    if h1['turno'] != h2['turno']:
        return False

    # 2. Verifica se há intersecção de dias da semana
    dias_em_comum = h1['dias'].intersection(h2['dias'])
    if not dias_em_comum:
        return False

    # 3. Verifica se há intersecção de blocos de horário
    blocos_em_comum = h1['blocos'].intersection(h2['blocos'])
    if not blocos_em_comum:
        return False

    # Se passou por todas as verificações, significa que há conflito
    return True


disciplinas = disc['id_disciplina'].tolist()
horarios = disc.set_index('id_disciplina')['horario'].to_dict()
num_disciplinas = len(disciplinas)
matriz_conflitos = pd.DataFrame(0, index=disciplinas, columns=disciplinas)


for i in range(num_disciplinas):
    for j in range(i, num_disciplinas):
        disc1 = disciplinas[i]
        disc2 = disciplinas[j]
        
        # Não compara uma disciplina com ela mesma
        if i == j:
            continue
            
        if verificar_conflito(horarios[disc1], horarios[disc2]):
            matriz_conflitos.loc[disc1, disc2] = 1
            matriz_conflitos.loc[disc2, disc1] = 1
matriz_conflitos.to_csv('dados/matriz_conflitos.csv', index=True)